## This note is for creating training sample for NER test.

In [345]:
import json
f1 = open("product_to_all.json")
f2 = open("main_categories_to_num.json")
f3 = open("labels_v2.json")
#get dictionary {"shoes": list[all categories]}
product_to_all = json.load(f1)
main_cat_to_num = json.load(f2)
original_labels_v2 = json.load(f3)

#### 1. Creat entity rules for all entities.

In [346]:
patterns_words= set()
with open("patterns_files/patterns_for_clothes.txt") as pa:
   patterns_words =  {l.lower().removesuffix('\n') for l in pa.readlines()}

In [289]:
colors_words = set()
with open("patterns_files/colors.txt") as colors_file:
    colors_words = {c.lower().removesuffix('\n') for c in colors_file.readlines()}
for o in original_labels_v2.get('color'):
    colors_words.add(o.lower().strip()) 

In [341]:
sizes_words = set()
with open("patterns_files/sizes.txt") as sizes_file:
    sizes_words = {s.replace('"', '').removesuffix(",\n") for s in sizes_file.readlines()}

In [348]:
#list all known patterns we need to detect.
shoes_words = product_to_all.get("shoes")
tops_words = product_to_all.get("tops")
bottoms_words = product_to_all.get("bottoms")
other_clothing_words = product_to_all.get("other_clothing")
beauty_words = product_to_all.get("beauty")
accessories_words = product_to_all.get("accessories")
homeware_words = product_to_all.get("homeware")
other_words = product_to_all.get("other")

genders_words = [o.lower().strip() for o in original_labels_v2.get("gender")]
main_cats_words = list(main_cat_to_num.keys())
colors_words = list(colors_words)
patterns_words = list(patterns_words)
sizes_words = list(sizes_words)

In [349]:
tops_pattern = [{'LOWER': {'in':tops_words}}]
shoes_pattern = [{'LOWER': {'in':shoes_words}}]
bottoms_pattern = [{'LOWER': {'in':bottoms_words}}]
other_clothing_pattern = [{'LOWER':{'in': other_clothing_words}}]
beauty_pattern =[{'LOWER': {'in': beauty_words}}]
accessories_pattern = [{'LOWER': {'in': accessories_words}}]
homeware_pattern = [{'LOWER' : {'in': homeware_words}}]
other_pattern = [{'LOWER': {'in': other_words}}]

genders_pattern = [{'LOWER': {'in': genders_words}}]
main_cats_pattern = [{'LOWER': {'in': main_cats_words}}]
colors_pattern = [{'LOWER': {'in': colors_words}}]
patterns_pattern = [{'LOWER': {'in': patterns_words}}]
sizes_pattern = [{'LOWER': {'in': sizes_words}}]


In [232]:
#intergate all patterns in one patterns list
all_patterns = []
all_patterns.append(tops_pattern)
all_patterns.append(shoes_pattern)
all_patterns.append(bottoms_pattern)
all_patterns.append(other_clothing_pattern)
all_patterns.append(beauty_pattern)
all_patterns.append(accessories_pattern)
all_patterns.append(homeware_pattern)
all_patterns.append(other_pattern)

all_patterns.append(genders_pattern)
all_patterns.append(main_cats_pattern)
all_patterns.append(colors_pattern)
all_patterns.append(patterns_pattern)
all_patterns.append(sizes_pattern)

In [327]:
import spacy
from spacy.lang.en import English
from spacy.pipeline import EntityRuler
nlp = spacy.load("en_core_web_lg")
entity_rulers = nlp.add_pipe("entity_ruler")


In [351]:
patterns_pattern

[{'LOWER': {'in': ['toile',
    'coin dot pattern',
    'half drop pattern',
    'striped',
    'celtic knot',
    'hairline pattern',
    'cheetah pattern',
    'liberty style',
    'foulard',
    'four-way layout',
    'crocodile pattern',
    'fleur-de-lis',
    "bull's eye",
    'pin check',
    'pinstriped ',
    'pin stripe',
    'allover pattern',
    'shepherd check',
    'retro',
    'glen checks',
    'lattice',
    'stripes ',
    'arabesque pattern',
    'straight repeat',
    'plaid ',
    'madras plaid',
    'geometric pattern',
    'basket weave pattern',
    'dress pattern',
    'unbalanced stripes',
    'argyle',
    'plain',
    'flame stitch',
    'zig zag',
    'ogee pattern',
    'adaptation pattern',
    "shepherd's check",
    'snake pattern',
    'herati',
    'leopard',
    'scroll pattern',
    'hexagonal',
    'two-way layout',
    'benmore',
    'gradation',
    'undirectional pattern',
    'christmas pattern',
    'tessellations',
    'designer',
    'disty

In [350]:
#product names
'''
entity_rulers.add_patterns([{'label':'TOPS', 'pattern': tops_pattern}])
entity_rulers.add_patterns([{'label':'BOTTOMS', 'pattern': bottoms_pattern}])
entity_rulers.add_patterns([{'label':'Other_CLOTHING', 'pattern': other_clothing_pattern}])
entity_rulers.add_patterns([{'label':"SHOES", 'pattern': shoes_pattern}])
entity_rulers.add_patterns([{'label':'HOMEWARE', 'pattern': homeware_pattern}])
entity_rulers.add_patterns([{'label':'ACCESSORIES', 'pattern': accessories_pattern}])
entity_rulers.add_patterns([{'label':'BEAUTY', 'pattern': beauty_pattern}])
entity_rulers.add_patterns([{'label':'OTHER', 'pattern': other_pattern}])
'''
#product attributes
entity_rulers.add_patterns([{'label':"GENDER", 'pattern': genders_pattern}])
entity_rulers.add_patterns([{'label':"MAIN_CATEGORIES", 'pattern':main_cats_pattern}])
entity_rulers.add_patterns([{'label':"COLOR", 'pattern':colors_pattern}])
entity_rulers.add_patterns([{'label':"PATTERN", 'pattern':patterns_pattern}])
entity_rulers.add_patterns([{'label':"SIZE", 'pattern':sizes_pattern}])

In [354]:
entity_rulers.add_patterns([{"label": "Gen", "pattern": "Black White"}])

In [355]:
doc = nlp("I have a Men black white watch. Also I have a plaid tops. ")
ents = [(ent.text, ent.label_) for ent in doc.ents]
ents

[('Men', 'GENDER'),
 ('black', 'COLOR'),
 ('white', 'COLOR'),
 ('plaid', 'PATTERN'),
 ('tops', 'MAIN_CATEGORIES')]

In [236]:
import pandas as pd
df = pd.read_csv("(Current)cleaned_dataset_v1.csv")

In [237]:
import sys
sys.path.append("/Users/luis/Documents/GitHub/2022Summer/AI-Oriented-Recommendation-System/PreprocessingData/")
from spaCy import entity_options as e

In [269]:

import random
from spacy import displacy
sample = df.loc[random.randint(0, df.shape[0]), "raw_text"]
doc = nlp(sample)
displacy.render(doc, style="ent", options=e.get_entity_options())

In [272]:
doc = nlp("Men black")
displacy.render(doc, style="ent",options=e.get_entity_options())